In [1]:
import sys
sys.path.append("../") # go to parent dir
import pandas as pd

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from omegaconf import OmegaConf
params = OmegaConf.load('../configs/predict_generator_config.yaml')

In [4]:
from datasets import load_from_disk
from datasets.filesystems import S3FileSystem

In [5]:
s3 = S3FileSystem()
dataset = load_from_disk(f's3://{params.bucket_name}/{params.output}', storage_options=s3.storage_options)

/Users/dmitriimaksimov/Desktop/GitHub/master_thesis_RAG_LLM/.venv/lib/python3.10/site-packages/fsspec/spec.py:79: FutureWarning: S3FileSystem is deprecated and will be removed in the next major version of datasets. Use s3fs.S3FileSystem instead.
  obj = super().__call__(*args, **kwargs)
/Users/dmitriimaksimov/Desktop/GitHub/master_thesis_RAG_LLM/.venv/lib/python3.10/site-packages/fsspec/registry.py:272: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


In [7]:
from src.evaluation.evaluation import generator_llm_evaluate

In [8]:
dataset['train'].select(range(10))

Dataset({
    features: ['question', 'ground_truth', 'page', 'document', 'contexts', 'distances', 'contexts_pages', 'contexts_documents', 'answer'],
    num_rows: 10
})

In [12]:
res = generator_llm_evaluate(dataset['train'].select(range(100)), 'gpt-3.5-turbo-1106', 1)

evaluating with [faithfulness]


  0%|          | 0/100 [00:00<?, ?it/s]

  7%|▋         | 7/100 [00:34<07:41,  4.96s/it]


KeyboardInterrupt: 

In [13]:
res

{'faithfulness': 0.7352}

In [15]:
res.scores

Dataset({
    features: ['faithfulness'],
    num_rows: 100
})

In [16]:
from datasets import concatenate_datasets

In [19]:
concatenate_datasets([dataset['train'].select(range(100)), res.scores], axis=1)

Dataset({
    features: ['question', 'ground_truth', 'page', 'document', 'contexts', 'distances', 'contexts_pages', 'contexts_documents', 'answer', 'faithfulness'],
    num_rows: 100
})

In [22]:
for a, j in res.items():
    print(a)
    print(j)

faithfulness
0.7351666666666666
